# Import Libraries and Data

In [439]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
pd.set_option("display.max_rows", 1000)

import sys
sys.path.append('../../scripts')

from html_parser import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [440]:
sentences = get_fiction_sentences("https://www.gutenberg.org/files/1342/1342-h/1342-h.htm")

In [465]:
sentences[100:110]

['"I hope Mr. Bingley will like it, Lizzy."',
 '"We are not in a way to know what Mr. Bingley likes," said her       mother resentfully, "since we are not to visit."',
 '"But you forget, mamma," said Elizabeth, "that we shall meet him at the       assemblies, and that Mrs. Long promised to introduce him."',
 '"I do not believe Mrs. Long will do any such thing. She has two nieces of       her own. She is a selfish, hypocritical woman, and I have no opinion of       her."',
 '"No more have I," said Mr. Bennet; "and I am glad to find that you do not       depend on her serving you."',
 'Mrs. Bennet deigned not to make any reply, but, unable to contain herself,       began scolding one of her daughters.',
 '"Don\'t keep coughing so, Kitty, for Heaven\'s sake! Have a little       compassion on my nerves. You tear them to pieces."',
 '"Kitty has no discretion in her coughs," said her father; "she times them       ill."',
 '"I do not cough for my own amusement," replied Kitty fretfully. "When

In [442]:
df = pd.DataFrame(sentences, columns=['text'])

index_to_exclude = [i for i in range(1,63)]
df = df.drop(index_to_exclude, axis=0)
df = df.reset_index()
df.shape

(2119, 2)

In [443]:
def get_num_quotes(sent):
    return sent.count('"')
    
def add_tag_to_dict(sent_index, sent, somedict, tag):
    global total_quotes_count
    num_utterance = int(get_num_quotes(sent)/2)
    somedict[sent_index] = (tag, total_quotes_count, num_utterance)
    return somedict
    
def is_quote_start_of_sent(sent):
    # must have capitalized alphabet after quote
    return bool(re.match('^"[A-Z]', sent))

def is_quote_end_of_sent(sent):
    return bool(re.match('.*"$', sent))

def tag_dialogue(sent_index, sent):
    global total_quotes_count, sent_dia_dict, sent_list
    
    num_quotes = get_num_quotes(sent)
    
    if num_quotes == 0:
#         if (total_quotes_count%2) == 0:
        add_tag_to_dict(sent_index, sent, sent_dia_dict, 'narrative')
#         else:
#             add_tag_to_dict(sent_index, sent, sent_dia_dict, 'dialogue-P')
    
    elif num_quotes == 1:
        total_quotes_count += 1
        add_tag_to_dict(sent_index, sent, sent_dia_dict, 'letter')#letter-S
#         if is_quote_start_of_sent(sent):
#             total_quotes_count += 1
#             add_tag_to_dict(sent_index, sent, sent_dia_dict, 'letter-S')
#         elif is_quote_end_of_sent(sent):
#             total_quotes_count += 1
#             add_tag_to_dict(sent_index, sent, sent_dia_dict, 'letter-E')
#         else:
#             if (total_quotes_count%2) == 0:
#                 total_quotes_count += 1
#                 add_tag_to_dict(sent_index, sent, sent_dia_dict, 'letter-S')
#             else: 
#                 total_quotes_count += 1
#                 add_tag_to_dict(sent_index, sent, sent_dia_dict, 'letter-E')
                
    elif num_quotes == 2:
#         if (total_quotes_count%2) == 0:
        total_quotes_count += 2
        add_tag_to_dict(sent_index, sent, sent_dia_dict, 'utterance')
#         else:
#             total_quotes_count += 2
#             add_tag_to_dict(sent_index, sent, sent_dia_dict, 'dialogue-E-S')
        
    elif num_quotes > 2:
#         if (total_quotes_count%2) == 0:
#             total_quotes_count += num_quotes
        if (num_quotes%2) == 0:
            total_quotes_count += num_quotes
            add_tag_to_dict(sent_index, sent, sent_dia_dict, 'utterance')
        else:
            total_quotes_count += num_quotes
            add_tag_to_dict(sent_index, sent, sent_dia_dict, 'letter')
#         else:
#             total_quotes_count += num_quotes
#             add_tag_to_dict(sent_index, sent, sent_dia_dict, 'dialogue-S')

    return sent_dia_dict

In [445]:
doc = df['text'].values

total_quotes_count = 0
sent_dia_dict = dict()
for sent_index, sent in enumerate(doc):
    sent_dia_dict = tag_dialogue(sent_index, sent)

In [446]:
len(df)

2119

In [448]:
df = pd.DataFrame(list(sent_dia_dict.items()), columns=['para_index', 'tag'])

In [449]:
df.shape

(2119, 2)

In [450]:
df['para'] = doc
df['num_quotes_up_to_para'] = df['tag'].apply(lambda x: x[1])
df['num_utterances'] = df['tag'].apply(lambda x: x[2])
df['tag'] = df['tag'].apply(lambda x: x[0])


In [451]:
df['tag'].value_counts()

utterance    1289
narrative    779 
letter       51  
Name: tag, dtype: int64

In [464]:
# df[(df['para'].apply(lambda x: is_quote_start_of_sent(x)==False)) & (df['tag']=='utterance')]

In [453]:
df.loc[100:200]

,para_index,tag,para,num_quotes_up_to_para,num_utterances
100,100,narrative,"The manner in which they spoke of the Meryton assembly was sufficiently characteristic. Bingley had never met with more pleasant people or prettier girls in his life; everybody had been most kind and attentive to him; there had been no formality, no stiffness; he had soon felt acquainted with all the room; and, as to Miss Bennet, he could not conceive an angel more beautiful. Darcy, on the contrary, had seen a collection of people in whom there was little beauty and no fashion, for none of whom he had felt the smallest interest, and from none received either attention or pleasure. Miss Bennet he acknowledged to be pretty, but she smiled too much.",186,0
101,101,narrative,"Mrs. Hurst and her sister allowed it to be so—but still they admired her and liked her, and pronounced her to be a sweet girl, and one whom they would not object to know more of. Miss Bennet was therefore established as a sweet girl, and their brother felt authorized by such commendation to think of her as he chose.",186,0
102,102,narrative,Chapter 5,186,0
103,103,narrative,"Within a short walk of Longbourn lived a family with whom the Bennets were particularly intimate. Sir William Lucas had been formerly in trade in Meryton, where he had made a tolerable fortune, and risen to the honour of knighthood by an address to the king during his mayoralty. The distinction had perhaps been felt too strongly. It had given him a disgust to his business, and to his residence in a small market town; and, in quitting them both, he had removed with his family to a house about a mile from Meryton, denominated from that period Lucas Lodge, where he could think with pleasure of his own importance, and, unshackled by business, occupy himself solely in being civil to all the world. For, though elated by his rank, it did not render him supercilious; on the contrary, he was all attention to everybody. By nature inoffensive, friendly, and obliging, his presentation at St. James's had made him courteous.",186,0
104,104,narrative,"Lady Lucas was a very good kind of woman, not too clever to be a valuable neighbour to Mrs. Bennet. They had several children. The eldest of them, a sensible, intelligent young woman, about twenty-seven, was Elizabeth's intimate friend.",186,0
105,105,narrative,That the Miss Lucases and the Miss Bennets should meet to talk over a ball was absolutely necessary; and the morning after the assembly brought the former to Longbourn to hear and to communicate.,186,0
106,106,utterance,"""You began the evening well, Charlotte,"" said Mrs. Bennet with civil self-command to Miss Lucas. ""You were Mr. Bingley's first choice.""",190,2
107,107,utterance,"""Yes; but he seemed to like his second better.""",192,1
108,108,utterance,"""Oh! you mean Jane, I suppose, because he danced with her twice. To be sure that did seem as if he admired her—indeed I rather believe he did—I heard something about it—but I hardly know what—something about Mr. Robinson.""",194,1
109,109,utterance,"""Perhaps you mean what I overheard between him and Mr. Robinson; did not I mention it to you? Mr. Robinson's asking him how he liked our Meryton assemblies, and whether he did not think there were a great many pretty women in the room, and which he thought the prettiest? and his answering immediately to the last question: 'Oh! the eldest Miss Bennet, beyond a doubt; there cannot be two opinions on that point.'""",196,1


In [454]:
def find_utterances_raw(para):
    quotes = re.findall('"([^"]*)"', para)
    return ['"' + i + '"' for i in quotes]

def remove_period_from_honorifics(para):
    regex = '(Miss.|Mrs.|Mr.|Ms.)'
    for i in re.findall(regex, para, flags=re.IGNORECASE):
        para = para.replace(i, i[:-1])
    return para

def tokenize_para(para):
    para = remove_period_from_honorifics(para)
#     regex = '((?![.\s])[^."]*(?:"[^"]*[^".]"[^."]*)*(?:"[^"]+\."|\.))'
    regex = '((?![.\s])[^."]*(?:"[^"]*[^".]"[^."]*)*(?:"[^"]+\."|\.|"[^"]+\?"|"[^"]+\!"))'
    return re.findall(regex, para)

In [455]:
teststring1 = '"My dear Mr. Bennet," replied his Ms. wife, "how can you be so tiresome! You must know that I am thinking of his marrying one of them."'
print(teststring1)

"My dear Mr. Bennet," replied his Ms. wife, "how can you be so tiresome! You must know that I am thinking of his marrying one of them."


In [456]:
tokenize_para(teststring1)

['"My dear Mr Bennet," replied his Ms wife, "how can you be so tiresome! You must know that I am thinking of his marrying one of them."']

In [457]:
find_utterances_raw(teststring1)

['"My dear Mr. Bennet,"',
 '"how can you be so tiresome! You must know that I am thinking of his marrying one of them."']

In [458]:
teststring2 = '"My dear Mr. Bennet," said his lady to him one day, "have you heard that Netherfield Park is let at last?"'
print(teststring2)

"My dear Mr. Bennet," said his lady to him one day, "have you heard that Netherfield Park is let at last?"


In [459]:
tokenize_para(teststring2)

['"My dear Mr Bennet," said his lady to him one day, "have you heard that Netherfield Park is let at last?"']

In [460]:
df['raw_utter_list'] = df['para'].apply(lambda x: find_utterances_raw(x))

In [461]:
df['tokenized_sent'] = df['para'].apply(lambda x: tokenize_para(x))

In [462]:
df.shape

(2119, 7)

In [463]:
df.head()

,para_index,tag,para,num_quotes_up_to_para,num_utterances,raw_utter_list,tokenized_sent
0,0,narrative,By Jane Austen,0,0,[],[]
1,1,narrative,Chapter 1,0,0,[],[]
2,2,narrative,"It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.",0,0,[],"[It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.]"
3,3,narrative,"However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.",0,0,[],"[However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.]"
4,4,utterance,"""My dear Mr. Bennet,"" said his lady to him one day, ""have you heard that Netherfield Park is let at last?""",4,2,"[""My dear Mr. Bennet,"", ""have you heard that Netherfield Park is let at last?""]","[""My dear Mr Bennet,"" said his lady to him one day, ""have you heard that Netherfield Park is let at last?""]"


In [501]:
df_labeled = pd.read_csv('../../data/parsed-n-labeled-data/pnp-gutenberg-label-task-pride-and-prejudice-by-jane-austen.csv', index_col=[0])

In [502]:
df_labeled = df_labeled.reset_index()
df_labeled.columns = ['para_index', 'tag_old','para', 'position', 'speaker', 'type']

In [503]:
df_labeled.shape

(2119, 6)

In [504]:
df_labeled.head()

,para_index,tag_old,para,position,speaker,type
0,0,not,By Jane Austen,n,NaN,NaN
1,1,not,Chapter 1,n,NaN,NaN
2,2,not,"It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.",n,NaN,NaN
3,3,not,"However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.",n,NaN,NaN
4,4,dialogue-F,"“My dear Mr. Bennet,” said his lady to him one day, “have you heard that Netherfield Park is let at last?”",1,mother,c


In [505]:
df_merged = pd.merge(df, df_labeled, on='para_index', how='left')

In [506]:
df_merged.shape

(2119, 12)

In [508]:
df_merged.tail()

,para_index,tag,para_x,num_quotes_up_to_para,num_utterances,raw_utter_list,tokenized_sent,tag_old,para_y,position,speaker,type
2114,2114,narrative,"Though Darcy could never receive him at Pemberley, yet, for Elizabeth's sake, he assisted him further in his profession. Lydia was occasionally a visitor there, when her husband was gone to enjoy himself in London or Bath; and with the Bingleys they both of them frequently staid so long, that even Bingley's good humour was overcome, and he proceeded so far as to talk of giving them a hint to be gone.",3531,0,[],"[Though Darcy could never receive him at Pemberley, yet, for Elizabeth's sake, he assisted him further in his profession., Lydia was occasionally a visitor there, when her husband was gone to enjoy himslf in London or Bath; and with the Bingleys they both of them frequently staid so long, that even Bingley's good humour was overcome, and he proceeded so far as to talk of giving them a hint to be gone.]",not,"Though Darcy could never receive him at Pemberley, yet, for Elizabeth's sake, he assisted him further in his profession. Lydia was occasionally a visitor there, when her husband was gone to enjoy himself in London or Bath; and with the Bingleys they both of them frequently staid so long, that even Bingley's good humour was overcome, and he proceeded so far as to talk of giving them a hint to be gone.",n,NaN,NaN
2115,2115,narrative,"Miss Bingley was very deeply mortified by Darcy's marriage; but as she thought it advisable to retain the right of visiting at Pemberley, she dropt all her resentment; was fonder than ever of Georgiana, almost as attentive to Darcy as heretofore, and paid off every arrear of civility to Elizabeth.",3531,0,[],"[MissBingley was very deeply mortified by Darcy's marriage; but as she thought it advisable to retain the right of visiting at Pemberley, she dropt all her resentment; was fonder than ever of Georgiana, almost as attentive to Darcy as heretofore, and paid off every arrear of civility to Elizabeth.]",not,"Miss Bingley was very deeply mortified by Darcy's marriage; but as she thought it advisable to retain the right of visiting at Pemberley, she dropt all her resentment; was fonder than ever of Georgiana, almost as attentive to Darcy as heretofore, and paid off every arrear of civility to Elizabeth.",n,NaN,NaN
2116,2116,narrative,"Pemberley was now Georgiana's home; and the attachment of the sisters was exactly what Darcy had hoped to see. They were able to love each other even as well as they intended. Georgiana had the highest opinion in the world of Elizabeth; though at first she often listened with an astonishment bordering on alarm at her lively, sportive, manner of talking to her brother. He, who had always inspired in herself a respect which almost overcame her affection, she now saw the object of open pleasantry. Her mind received knowledge which had never before fallen in her way. By Elizabeth's instructions, she began to comprehend that a woman may take liberties with her husband which a brother will not always allow in a sister more than ten years younger than himself.",3531,0,[],"[Pemberley was now Georgiana's home; and the attachment of the sisters was exactly what Darcy had hoped to see., They were able to love each other even as well as they intended., Georgiana had the highest opinion in the world of Elizabeth; though at first she often listened with an astonishment bordering on alarm at her lively, sportive, manner of talking to her brother., He, who had always inspired in herself a respect which almost overcame her affection, she now saw the object of open pleasantry., Her mind received knowledge which had never before fallen in her way., By Elizabeth's instructions, she began to comprehend that a woman may take liberties with her husband which a brother will not always allow in a sister more than ten years younger than himslf.]",not,"Pemberley was now Georgiana's home; and the attachment of the sisters was exactly what D

In [466]:
 s = df.xs(3)

In [467]:
s

para_index               3                                                                                                                                                                                                                                                                                    
tag                      narrative                                                                                                                                                                                                                                                                            
para                     However little known the feelings or views of such a man may be on his       first entering a neighbourhood, this truth is so well fixed in the minds       of the surrounding families, that he is considered the rightful property       of some one or other of their daughters.  
num_quotes_up_to_para    0                                                                 

In [480]:
import more_itertools
tokenized_sent_list = list(more_itertools.flatten(df_merged['tokenized_sent'].values))

In [481]:
len(tokenized_sent_list)

3713

In [520]:
{}

TypeError: dict expected at most 1 arguments, got 2

In [530]:
new_label_dict = dict()
for row in df_merged.index:
    new_label_dict[row] = [(i , df_merged.loc[row]['position']) for i in df_merged.loc[row]['tokenized_sent']]


In [531]:
new_label_dict

{0: [],
 1: [],
 2: [('It is a truth universally acknowledged, that a single man in possession of       a good fortune, must be in want of a wife.',
   'n')],
 3: [('However little known the feelings or views of such a man may be on his       first entering a neighbourhood, this truth is so well fixed in the minds       of the surrounding families, that he is considered the rightful property       of some one or other of their daughters.',
   'n')],
 4: [('"My dear Mr Bennet," said his lady to him one day, "have you heard that       Netherfield Park is let at last?"',
   '1')],
 5: [('Mr Bennet replied that he had not.', 'n')],
 6: [('"But it is," returned she; "for Mrs Long has just been here, and she told       me all about it."',
   '1')],
 7: [('Mr Bennet made no answer.', 'n')],
 8: [('"Do you not want to know who has taken it?" cried his wife impatiently.',
   '1')],
 9: [('"You want to tell me, and I have no objection to hearing it."', '2')],
 10: [('This was invitation enough.'